## Type hints

Introduced in Python 3.6 and gradually improved.

Unlike in other languages (Java, C++), they have NO EFFECT on the execution, are used by the developers and the editor to reduce mistakes.



In [9]:
# note you'd actually use:
# import re
# print(len(re.findall(r'(?-i:[aeiou])', 'ciao')))

def count_vowels(txt: str) -> int:
    total: int = 0
    for char in txt:
        if char in 'aeiouAEIOU':
            total += 1
    return total
            

In [14]:
print(count_vowels("hello"))
# this works even though the type is wrong
# and the comparison gives wrong results
print(count_vowels(['hel', 'lo']))

2
0


This is a notebook and for simplicity we are not going to run mypy or ruff here.

It is possible using tools like [nbQA](https://github.com/nbQA-dev/nbQA) but in general notebooks are to show analysis, do experiments or take notes, not to develop.

Placing the code above in a python file and invoking mypy on it we get:
```bash
$ mypy snippets/test_mypy.py 
snippets/test_mypy.py:11: error: Argument 1 to "count_vowels" has incompatible type "list[str]"; expected "str"  [arg-type]
Found 1 error in 1 file (checked 1 source file)

```

Type checking can be integrated with the IDE (both VSCode and Pycharm have extensions for it, even vim/helix/neovim can do it) to give you warnings **as you type**.

Python allows for **incremental typing**, meaning you can add type hints gradually, like you would in a language like Typescript.

Mypy has a strict mode (`mypy --strict myfile.py`) which enforces having types on every single function. It's quite a rigid requirement so it's not the default behavior.

Due to the dynamic nature of Python you often have to apply **type erasure**:

In [22]:
class Circle:
    def __init__(self, radius: float):
        self.type = 'circle'
        self.radius = radius

class Square:
    def __init__(self, side: float):
        self.type = 'square'
        self.side = side

def get_area(s: Square|Circle) -> float:
    if s.type == 'circle':
        assert isinstance(s, Circle)
        return s.radius ** 2 * 3.14
    else:
        assert isinstance(s, Square)
        return s.side ** 2

print(get_area(Circle(1.2)))
print(get_area(Square(1.2)))


4.5216
1.44


In this example we add `assert` when we know something about the type that mypy cannot guess. The presence of the assert gives it more details about the code coming after, becaus it know what's the type of `s` in that point.

This also works:


In [24]:
def get_area2(s: Square|Circle) -> float:
    if isinstance(s, Circle):
        return s.radius ** 2 * 3.14
    elif isinstance(s, Square):
        return s.side ** 2
    else:
        raise TypeError(f"Received object {s} of unknown type {type(s)}")

In this case mypy "understands" that the code inside the if can be reached only if `s` has a specific type and is valid.

This variation has the added benefit of breaking if in the future is called with a class that we don't know, showing a clear error.